In [ ]:

learning_rate=0.01
batch_size=100
display_step = 1
mnist = input_data.read_data_sets('mnist_data/',one_hot=True)

X=tf.placeholder(dtype=tf.float32,shape=[None,784], name='X')
Y=tf.placeholder(dtype=tf.float32,shape=[None,10], name='Y')

W = tf.Variable(tf.zeros([784,10]),name='W')
b= tf.Variable(tf.zeros([10]),name='b')

with tf.name_scope('linear'):
    Z=tf.matmul(X,W) +b

with tf.name_scope('softmax'):
    A=tf.nn.softmax(Z)

with tf.name_scope('cost'):
    cost = tf.reduce_mean(-tf.reduce_sum(Y*tf.log(A), axis=-1))

with tf.name_scope('train'):    
    train_step = tf.train.GradientDescentOptimizer(learning_rate).minimize(cost)

init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    batch_index=0
    total_batch = int(mnist.train.num_examples/batch_size)
    with tf.name_scope('fit_with_training_data'):
        for i in range(total_batch):
            batch_xs, batch_ys = mnist.train.next_batch(batch_size)
            _, batch_cost = sess.run([train_step, cost], feed_dict={X:batch_xs,Y:batch_ys}) 
            batch_index +=1
        #print("Batch:", '%04d' % (batch_index), "cost=", "{:.9f}".format(batch_cost))                        
            
    print("Optimization Finished!")     
    print(str(W))
    print(str(b))
    
    with tf.name_scope('calculate_accuracy'):
        # Test model
        correct_prediction = tf.equal(tf.argmax(A, 1), tf.argmax(Y, 1))
        # Calculate accuracy
        accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

        print("Train Accuracy:", accuracy.eval({X: mnist.train.images, Y: mnist.train.labels}))
        print("Test Accuracy:", accuracy.eval({X: mnist.test.images, Y: mnist.test.labels}))
    
    with tf.name_scope('single_image_test'):
        image=mnist.train.images[600]
        result=sess.run(tf.argmax(A,axis=1),feed_dict={X:image.reshape(1,784)})
        print("The recognized digit is {}".format(result))
        plt.imshow(image.reshape(28,28))
    
writer = tf.summary.FileWriter('./tbout/mnist',sess.graph)
writer.close()    